In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import urllib
import re
import pandas as pd
import itertools
import numpy as np
import multiprocessing as mp
import time

In [2]:
manager = mp.Manager()
projectlist = manager.list()

In [3]:
from multiprocessing import Pool
from joblib import Parallel, delayed

In [4]:
def remove(lst):
    remove_lst = ['',' ','  ','Project Details','Source(s) of financing']
    llst = []
    for i in lst:
        if i not in remove_lst:
            llst.append(i)
    return llst
    

In [ ]:
# def get_index(lst):
    temp = [] 
    for i in ['Objectives','Description','Benefits']:
        try:
            index = lst.index(i)
            temp.append(index)
        except: pass
    return temp
    
    

# Without Multi-processing

In [ ]:
global_url = "https://www.afdb.org/" 
df_null = pd.DataFrame()

In [7]:
url_list = ['https://www.afdb.org/en/projects-and-operations']
for num in range(1,221):
    url_list.append('https://www.afdb.org/en/projects-and-operations?page='+str(num))

In [ ]:
url_list = ['https://www.afdb.org/en/projects-and-operations']
for num in range(1,221):
    url_list.append('https://www.afdb.org/en/projects-and-operations?page='+str(num))

for url in url_list:
    try: 
        uClient = uReq(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        links = [a['href'] for a in page_soup.find_all('a', href=True)]
    
        drop_link = []
        for link in links: 
            if '/en/projects-and-operations/p-' in link:
                drop_link.append(link)
        drop_link = list(set(drop_link))
    
        for link in drop_link:
            bluClient = uReq(global_url+link)
            blupage_html = bluClient.read()
            bluClient.close()
            blupage_soup = soup(blupage_html, "html.parser")
       
            lst = blupage_soup.find_all('td')[0].text.split('\n')
            res = [i for i in lst if i] 
            llst = remove(res)
  
            dic = {}
            temp = get_index(llst)
            if len(temp)>0:
                string = llst[0:temp[0]-1]
                for i in string:
                    substr = i.split(': ')
                    dic[substr[0]] = substr[1]
                for i in temp:
                    dic[llst[i]]=llst[i+1]
            else:
                string = llst[0:]
                for i in string:
                    substr = i.split(': ')
                    dic[substr[0]] = substr[1]
            df = pd.DataFrame.from_dict(dic,orient="index").T
            df_null = pd.concat([df_null,df],sort=False)
    except:
            print(llst)
            
    print(url)


# With Multi-Processing

In [8]:
def pickle_creating(url):
    global projectlist
    try: 
        uClient = uReq(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        links = [a['href'] for a in page_soup.find_all('a', href=True)]
    
        drop_link = []
        for link in links: 
            if '/en/projects-and-operations/p-' in link:
                drop_link.append(link)
        drop_link = list(set(drop_link))
    
        for link in drop_link:
            bluClient = uReq(global_url+link)
            blupage_html = bluClient.read()
            bluClient.close()
            blupage_soup = soup(blupage_html, "html.parser")
       
            lst = blupage_soup.find_all('td')[0].text.split('\n')
            res = [i for i in lst if i] 
            llst = remove(res)
  
            dic = {}
            temp = get_index(llst)
            if len(temp)>0:
                string = llst[0:temp[0]-1]
                for i in string:
                    substr = i.split(': ')
                    dic[substr[0]] = substr[1]
                for i in temp:
                    dic[llst[i]]=llst[i+1]
            else:
                string = llst[0:]
                for i in string:
                    substr = i.split(': ')
                    dic[substr[0]] = substr[1]
            projectlist.append(dic)
            
    except:
           pass
    print('done')

In [9]:
num_cores=mp.cpu_count()
Parallel(n_jobs=num_cores)(delayed(pickle_creating)(j) for j in url_list)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
pd.DataFrame.from_dict(list(projectlist)).set_index('ID')

,ACFA,ADF,AWF,AfDB,Approval date,Benefits,Cofinanced,Country,Currency,Description,...,NTF,Name,OPEC,Objectives,RWSSI,SRF,Sector,Status,Task Manager,Total cost
ID,,,,,,,,,,,,,,,,,,,,,
P-CM-F00-006,NaN,NaN,NaN,102245467.8,14-Dec-2018,NaN,29493884.93,Cameroon,UAC,NaN,...,NaN,NACHTIGAL HYDRO POWER PROJECT,NaN,NaN,NaN,NaN,Power,Completed,"JALARD Matthieu Benoit, PESR2",786500000
P-CI-FA0-017,NaN,NaN,NaN,35741136.52,07-Nov-2018,NaN,NaN,Côte D'Ivoire,UAC,NaN,...,NaN,PROJET D'AMELIORATION DE L'ACCES A L'ELECTRICI...,NaN,NaN,NaN,NaN,Power,Approved,"KITANDALA LUHANA Raymond, RDGW1",50000000
P-Z1-FA0-162,NaN,NaN,NaN,21925580.4,14-Dec-2018,NaN,NaN,Multinational,UAC,NaN,...,NaN,SUPPLEMENTARY FINANCING,NaN,NaN,NaN,NaN,Power,Approved,"WUBESHET ZEGEYE Alemayehu, RDGE1",21925625.7
P-SN-F00-009,NaN,NaN,NaN,43751013.55,27-Nov-2018,NaN,NaN,Senegal,UAC,NaN,...,NaN,MALICOUNDA 120 MW HFO IPP,NaN,NaN,NaN,NaN,Power,Approved,"ABEDIER Charles Wilfried, PESR2",NaN
P-LR-HAB-002,NaN,NaN,NaN,3446849.58,14-Dec-2018,NaN,2274920.72,Liberia,UAC,NaN,...,NaN,TRADE FINANCE LINE OF CREDIT INTERNATIONAL BAN...,NaN,NaN,NaN,NaN,Finance,Approved,"BA Mohamadou, PIFD",3452250
P-GN-A00-004,NaN,NaN,NaN,NaN,21-Nov-2018,NaN,7870000,Guinea,UAC,NaN,...,NaN,PROJET D’APPUI À LA TRANSFORMATION DE L’AGRICU...,NaN,NaN,NaN,NaN,Agriculture,Approved,"ATTIOGBEVI-SOMADO Eklou, RDGW2",7870000
P-NG-F00-020,NaN,NaN,NaN,105327463.1,29-Nov-2018,"Project beneficiaries will be households, micr...",421309852.3,Nigeria,UAC,"This project, implemented by the Rural Electri...",...,NaN,NIGERIA ELECTRIFICATION PROJECT,NaN,To increase access to electricity services for...,NaN,NaN,Power,Approved,"MSHANA Rhoda Limbani, RDNG",526635000
P-BJ-DB0-017,NaN,NaN,NaN,99500000,13-Dec-2018,NaN,56300000,Benin,UAC,NaN,...,NaN,PROGRAMME D'AMENAGEMENT ROUTE COTONIERES,NaN,NaN,NaN,NaN,Transport,Approved,"DIOP Maimounatou Ndiaye, RDGW3",165000000
P-Z1-KE0-023,NaN,NaN,NaN,71570178.14,14-Dec-2018,NaN,24154935.12,Multinational,UAC,NaN,...,NaN,EMERGING AFRICA INFRASTRUCTURE FUND PROJECT (E...,NaN,NaN,NaN,NaN,Multi-Sector,Approved,"BEN AYED Hassen, PICU4",607843500
